##**Installations**

In [ ]:
%%capture
!unzip /content/drive/MyDrive/ASR.zip

In [ ]:
%%capture
# !pip show kenlm
import os

!pip install fairseq
!pip install 'https://github.com/kpu/kenlm/archive/master.zip'
!pip install pykaldi

os.environ['FAIRSEQ_ROOT'] = ' /usr/local/lib/python3.7/dist-packages/fairseq'
os.environ['KALDI_ROOT'] = ' /usr/local/lib/python3.7/dist-packages/kaldi'
os.environ['KENLM_ROOT'] = ' /usr/local/lib/python3.7/dist-packages/kenlm'

!git clone 'https://github.com/zhenghuatan/rVADfast.git'
os.environ['RVAD_ROOT'] = '/content/rVADfast'

In [ ]:
%%capture
%mkdir /content/trial/
%mkdir /content/trial/audio/
%mkdir /content/trial/gen/
!tar -xvzf /content/drive/MyDrive/dev-clean.tar.gz -C /content/trial/audio/

##**LibriSpeech**

Create Training and Validation Split from aw Audio

In [ ]:
!python /usr/local/lib/python3.7/dist-packages/fairseq/examples/wav2vec/wav2vec_manifest.py /content/trial/audio/LibriSpeech/dev-clean/ --ext 'flac' --dest /content/trial/ --valid-percent 0.95

In [ ]:
%cd ASR

Voice activity detection to remove silence

In [ ]:
# df=pd.read_csv('/content/trial/train.tsv', delimiter='\t')
# df.to_csv('/content/trial/train.tsv', index=False, sep='\t')
!python scripts/vads.py -r '/content/rVADfast' <'/content/trial/train.tsv' >'train.vads'

Remove silence

In [ ]:
!python scripts/remove_silence.py --tsv '/content/trial/train.tsv' --vads 'train.vads' --out  '/content/trial/gen/'

Download wav2vec2.0 model

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

In [ ]:
# print(model)
torch.save(model.state_dict(), '/content/trial/model.pt')

Preprocess audio

In [ ]:
%mkdir /content/trial/output_int1/
!bash /content/ASR/scripts/prepare_audio.sh /content/trial /content/trial/output_int1 /content/trial/model.pt 512 14

Preprocess words to phonemes as true samples to feed the generator

In [ ]:
%mkdir /content/trial/words
!bash scripts/prepare_text.sh english /content/drive/MyDrive/file.txt /content/trial/words/ 1000 espeak /content/drive/MyDrive/lid.176.bin

#**TIMIT**
Load and preprocess

In [ ]:
%mkdir /content/TIMIT/
!unzip /content/drive/MyDrive/TIMIT.zip -d /content/TIMIT/

In [ ]:
%mkdir /content/timitop

In [ ]:
!pwd

In [ ]:
!bash /content/ASR/scripts/prepare_timit.sh '/content/TIMIT/data/lisa/data/timit/raw/TIMIT/'  '/content/timitop/' '/content/model.pt'